In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

In [4]:
df=pd.read_csv('Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv')


In [5]:
df.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,22,1266342,41,44,2664,6954,456,0,64.975610,109.864573,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,22,1319353,41,44,2664,6954,456,0,64.975610,109.864573,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,22,160,1,1,0,0,0,0,0.000000,0.000000,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,22,1303488,41,42,2728,6634,456,0,66.536585,110.129945,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,35396,77,1,2,0,0,0,0,0.000000,0.000000,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [6]:
df.columns


Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', '

In [ ]:
encoder = LabelEncoder()
df[' Label']= encoder.fit_transform(df[' Label'])

In [8]:
df.head()


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,22,1266342,41,44,2664,6954,456,0,64.975610,109.864573,...,32,0.0,0.0,0,0,0.0,0.0,0,0,0
1,22,1319353,41,44,2664,6954,456,0,64.975610,109.864573,...,32,0.0,0.0,0,0,0.0,0.0,0,0,0
2,22,160,1,1,0,0,0,0,0.000000,0.000000,...,32,0.0,0.0,0,0,0.0,0.0,0,0,0
3,22,1303488,41,42,2728,6634,456,0,66.536585,110.129945,...,32,0.0,0.0,0,0,0.0,0.0,0,0,0
4,35396,77,1,2,0,0,0,0,0.000000,0.000000,...,32,0.0,0.0,0,0,0.0,0.0,0,0,0


In [9]:
df[' Label'].value_counts().sum


<bound method Series.sum of  Label
1    158930
0    127537
Name: count, dtype: int64>

In [10]:
df = df.fillna(0)  # Replace NaN with 0
df = df.replace([np.inf, -np.inf], 0)

In [11]:
df.isnull().sum()


 Destination Port              0
 Flow Duration                 0
 Total Fwd Packets             0
 Total Backward Packets        0
Total Length of Fwd Packets    0
                              ..
Idle Mean                      0
 Idle Std                      0
 Idle Max                      0
 Idle Min                      0
 Label                         0
Length: 79, dtype: int64

In [12]:
df=df.astype(int)


In [ ]:
X = df.drop(' Label',axis=1)
y = df[' Label']

In [ ]:
# Impute missing values (replace NaNs with the mean)
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Determine the number of columns (features) in your DataFrame
num_columns = df.shape[1]

# Set an appropriate value for k (less than or equal to the number of columns)
k = min(10, num_columns)  # Adjust this as needed

# Initialize SelectKBest with the scoring function
k_best = SelectKBest(score_func=f_classif, k=k)

# Fit and transform the imputed data to select the top 10 features
X_new = k_best.fit_transform(X_imputed, y)

In [15]:
# Get the boolean mask of selected features
selected_features_mask = k_best.get_support()



In [16]:
elected_feature_names = X.columns[selected_features_mask]


In [17]:
elected_feature_names

Index([' Fwd Packet Length Min', ' Bwd Packet Length Min',
       ' Bwd Packet Length Mean', ' Min Packet Length', ' Packet Length Mean',
       ' PSH Flag Count', ' ACK Flag Count', ' Average Packet Size',
       ' Avg Bwd Segment Size', ' min_seg_size_forward'],
      dtype='object')

In [18]:
new_columns=[' Destination Port', ' Bwd Packet Length Min',
       ' Bwd Packet Length Mean', ' Bwd Packets/s', ' Min Packet Length',
       ' PSH Flag Count', ' URG Flag Count', ' Avg Fwd Segment Size',
       ' Avg Bwd Segment Size', ' min_seg_size_forward']

In [19]:
df_new=X[new_columns]

In [20]:
df_new

,Destination Port,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packets/s,Min Packet Length,PSH Flag Count,URG Flag Count,Avg Fwd Segment Size,Avg Bwd Segment Size,min_seg_size_forward
0,22,0,158,34,0,1,0,64,158,32
1,22,0,158,33,0,1,0,64,158,32
2,22,0,0,6250,0,0,1,0,0,32
3,22,0,157,32,0,1,0,66,157,32
4,35396,0,0,25974,0,0,1,0,0,32
...,...,...,...,...,...,...,...,...,...,...
286462,443,0,1856,290,0,1,0,27,1856,20
286463,443,0,1769,155,0,1,0,27,1769,20
286464,443,0,2096,636,0,1,0,20,2096,20
286465,443,0,1777,433,0,1,0,26,1777,20


In [ ]:
df_new['label']=df[' Label']

In [22]:
X1=df_new.iloc[:,:-1].values
y1=df_new.iloc[:,-1].values

In [23]:
X1

array([[  22,    0,  158, ...,   64,  158,   32],
       [  22,    0,  158, ...,   64,  158,   32],
       [  22,    0,    0, ...,    0,    0,   32],
       ...,
       [ 443,    0, 2096, ...,   20, 2096,   20],
       [ 443,    0, 1777, ...,   26, 1777,   20],
       [ 443,    0, 1755, ...,   28, 1755,   20]])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.3, random_state=42)

In [26]:
ann = tf.keras.models.Sequential()

In [27]:
ann.add(tf.keras.layers.Dense(units=10,activation='sigmoid'))

In [28]:
ann.add(tf.keras.layers.Dense(units=10,activation='sigmoid'))

In [29]:
ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

In [30]:
ann.compile(optimizer ='adam' , loss='binary_crossentropy' , metrics= ['accuracy'])

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

In [32]:
#training the ann on training set
ann.fit(X_train,y_train,batch_size=55,epochs=12,callbacks=[early_stopping])

Epoch 1/12
3646/3646 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.8350 - loss: 0.3710
Epoch 2/12
 119/3646 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.9530 - loss: 0.1545

C:\Users\Daniel Holz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\callbacks\early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


3646/3646 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9537 - loss: 0.1450
Epoch 3/12
3646/3646 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9548 - loss: 0.1374
Epoch 4/12
3646/3646 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.9535 - loss: 0.1448
Epoch 5/12
3646/3646 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9552 - loss: 0.1371
Epoch 6/12
3646/3646 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9564 - loss: 0.1356
Epoch 7/12
3646/3646 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9560 - loss: 0.1332
Epoch 8/12
3646/3646 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9567 - loss: 0.1366
Epoch 9/12
3646/3646 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9557 - loss: 0.1374
Epoch 10/12
3646/3646 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9563 - loss: 0.1406
Epoch 11/12
3646/3646 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9551 - loss: 0.1352
Epoch 12/12
3646/3646 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.9554 - loss: 0.1376


In [33]:
df_new[df[' Label']==1].head(5) # portscan

,Destination Port,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packets/s,Min Packet Length,PSH Flag Count,URG Flag Count,Avg Fwd Segment Size,Avg Bwd Segment Size,min_seg_size_forward,label
1463,80,0,282,0,0,1,0,117,282,32,1
1467,80,0,0,14285,0,0,1,0,0,32,1
1468,80,0,282,0,0,1,0,117,282,32,1
1480,80,0,0,19230,0,0,1,0,0,32,1
1511,80,0,0,0,0,1,0,0,0,32,1


In [34]:
df_new[df[' Label']==0].head(5) # benign

,Destination Port,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packets/s,Min Packet Length,PSH Flag Count,URG Flag Count,Avg Fwd Segment Size,Avg Bwd Segment Size,min_seg_size_forward,label
0,22,0,158,34,0,1,0,64,158,32,0
1,22,0,158,33,0,1,0,64,158,32,0
2,22,0,0,6250,0,0,1,0,0,32,0
3,22,0,157,32,0,1,0,66,157,32,0
4,35396,0,0,25974,0,0,1,0,0,32,0


In [ ]:
# Valores hipotéticos para as 10 features selecionadas
input_data_1 = np.array([[80, 0, 282, 0, 0, 1, 0, 117, 282, 32]])  # label 1 (Portscan)
input_data_1_1 = np.array([[80, 0, 0, 14285, 0, 0, 1, 0, 0, 32]])  # label 1 (Portscan)

input_data_0 = np.array([[35396, 0, 0, 25974, 0, 0, 1, 0, 0, 32]])  # label 0 (Benigno)
input_data_0_1 = np.array([[22, 0, 0, 6250, 0, 0, 1, 0, 0, 32]])  # label 0 (Benigno)

# Definir o limiar de decisão
threshold = 0.5

# Previsões com limiar aplicado
prediction_1 = (ann.predict(input_data_1) > threshold).astype(int)
prediction_1_1 = (ann.predict(input_data_1_1) > threshold).astype(int)
prediction_0 = (ann.predict(input_data_0) > threshold).astype(int)
prediction_0_1 = (ann.predict(input_data_0_1) > threshold).astype(int)

# Exibir as previsões (Portscan=1, Benigno=0)
print(f'Prediction 1 (input_data_1): {"Portscan" if prediction_1 == 1 else "Benigno"}')
print(f'Prediction 1_1 (input_data_1_1): {"Portscan" if prediction_1_1 == 1 else "Benigno"}')
print(f'Prediction 0 (input_data_0): {"Portscan" if prediction_0 == 1 else "Benigno"}')
print(f'Prediction 0_1 (input_data_0_1): {"Portscan" if prediction_0_1 == 1 else "Benigno"}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Prediction 1 (input_data_1): Benigno
Prediction 1_1 (input_data_1_1): Portscan
Prediction 0 (input_data_0): Portscan
Prediction 0_1 (input_data_0_1): Portscan


In [38]:
# Save the trained model in the native Keras format
ann.save('ann_model.h5')
